In [5]:
''' Google検索トップ10の結果(H1,H2,H3タグ)を取得しエクセルに出力するツール
    https://fastclassinfo.com/entry/python_googlesearchanalysis_eachpage/ '''

from datetime import datetime
import os
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import openpyxl
from openpyxl.styles import Font
import time
import requests
from bs4 import BeautifulSoup

def enable_download_in_headless_chrome(browser, download_dir):
    browser.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
    params = {'cmd': 'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath': download_dir}}
    browser.execute("send_command", params)

def main():
    print('検索キーワードを入力してください')
    search_query=input()

    options = Options()
    options.add_argument('--headless')

    path = os.getcwd()
    exefile = path + "\\chromedriver.exe"
    driver = webdriver.Chrome(options=options, executable_path=exefile)

    driver.implicitly_wait(10)    # 暗黙的な待機
    
    enable_download_in_headless_chrome(driver, path)
    driver.get('https://www.google.co.jp/search?q='+search_query)

    title=[]
    url=[]
    wb=openpyxl.Workbook()
    ws=wb.active
    font = Font(color='0000CC', underline='single')
    
    ws.cell(row=1, column=1).value = "キーワード"
    ws.cell(row=1, column=2).value = search_query

    title_header=["順位", "タイトル" , "H1" , "H2" , "H3"]
    for a in range(len(title_header)):
        b=int(a)
        ws.cell(row=3, column=b+1).value = title_header[b]

    lastrow = 4

    for i, g in enumerate(driver.find_elements_by_class_name("g")):
        r = g.find_element_by_class_name("r")
        title.append(r.find_element_by_tag_name("h3").text)  # タイトル
        url.append(r.find_element_by_tag_name("a").get_attribute("href"))  # URL

    for k in range(len(url)): #len(url) 
        q=int(k)
        ws.cell(row=lastrow, column=1).value = q+1
        ws.cell(row=lastrow, column=2).value = title[q]
        ws.cell(row=lastrow, column=2).hyperlink = url[q]
        ws.cell(row=lastrow, column=2).font = font
        lastrow=lastrow+1
        reqs = requests.get(url[q])
        soup = BeautifulSoup(reqs.text, 'lxml')
        for heading in soup.find_all(["h1", "h2", "h3"]):
            if heading.name == "h1":
                ws.cell(row=lastrow, column=3).value = heading.text.strip()
                lastrow=lastrow+1
            elif heading.name == "h2":
                ws.cell(row=lastrow, column=4).value = heading.text.strip()
                lastrow=lastrow+1
            else:
                ws.cell(row=lastrow, column=5).value = heading.text.strip()
                lastrow=lastrow+1

    Excelfile = "GoogleSearch_" + search_query + "_" + datetime.now().strftime("%Y%m%d_%H%M%S") + ".xlsx"
    saveFilePath=path +"\\" + Excelfile
    wb.save(saveFilePath)

if __name__ == "__main__":
    main()

検索キーワードを入力してください
paiza


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"h3"}
  (Session info: headless chrome=87.0.4280.66)
